# Singapore Attraction Discovery Agent
Agentic system for discovering Singapore attractions using tool-based reasoning, LLM decision-making, and iterative refinement to meet user requirements.

In [1]:
# Install requirements if needed
# !pip install -r requirements.txt

In [2]:
# Imports
import os
import sys
import json
import logging
from typing import Dict, List, Any, Optional
from dotenv import load_dotenv

# Add src to path
sys.path.append(os.path.join(os.path.dirname('.'), 'src'))

from src.singapore_places_api import SingaporePlacesAPI
from src.transport_analyzer import SingaporeTransportAnalyzer
from src.carbon_calculator import SingaporeCarbonCalculator
from src.place_relationship_graph import PlaceGraphGenerator

try:
    from openai import OpenAI
except ImportError:
    OpenAI = None

try:
    from anthropic import Anthropic
except ImportError:
    Anthropic = None

In [3]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('agent_reasoning.log', mode='a')
    ]
)
logger = logging.getLogger(__name__)

In [4]:
# Load environment variables
load_dotenv()

# Check API keys
print("API Keys Status:")
print(f"GOOGLE_MAPS_API_KEY: {'✓ Set' if os.getenv('GOOGLE_MAPS_API_KEY') else '✗ Missing'}")
print(f"CLIMATIQ_API_KEY: {'✓ Set' if os.getenv('CLIMATIQ_API_KEY') else '✗ Missing'}")
print(f"OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Missing'}")
print(f"ANTHROPIC_API_KEY: {'✓ Set' if os.getenv('ANTHROPIC_API_KEY') else '✗ Missing'}")

API Keys Status:
GOOGLE_MAPS_API_KEY: ✓ Set
CLIMATIQ_API_KEY: ✓ Set
OPENAI_API_KEY: ✓ Set
ANTHROPIC_API_KEY: ✗ Missing


## Input Data Loading
Load and validate input file with trip requirements.

In [5]:
def load_input_file(file_path: str) -> dict:
    """Load and validate input file."""
    try:
        with open(file_path, 'r') as f:
            input_data = json.load(f)
        
        # Basic validation
        required_fields = ["trip_dates", "duration_days", "budget", "pace"]
        for field in required_fields:
            if field not in input_data:
                raise ValueError(f"Missing required field: {field}")
        
        # Check if optional section exists, and validate accommodation_location if it does
        if "optional" in input_data and "accommodation_location" not in input_data["optional"]:
            raise ValueError("Missing required field: optional.accommodation_location")
        
        return input_data
    
    except FileNotFoundError:
        print(f"Error: Input file '{file_path}' not found.")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON in input file: {e}")
        return None
    except ValueError as e:
        print(f"Error: {e}")
        return None

# Test with different input files
print("Available input files:")
for file in os.listdir('inputs'):
    if file.endswith('.json'):
        print(f"  inputs/{file}")

Available input files:
  inputs/adventure_input.json
  inputs/complex_input.json
  inputs/family_budget_input.json
  inputs/garden_only_input.json
  inputs/simple_input.json


## Places Research Agent - Focused Implementation
Handles location-based attraction discovery using Google Places API

In [ ]:
from tools import search_places, search_multiple_keywords, get_place_details

class PlacesResearchAgent:
    """
    Specialized agent for location-based attraction research using Google Places API.
    """

    def __init__(self, system_prompt=""):
        # Initialize OpenAI client
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        
        # Agent configuration
        self.model_name = "gpt-4o"
        self.temperature = 0.3
        
        # Message history
        self.messages = []
        self.system_prompt = system_prompt or self._get_default_system_prompt()
        
        if self.system_prompt:
            self.messages.append({"role": "system", "content": self.system_prompt})
        
        # Tool definitions for Google Places API
        self.available_tools = self._define_places_tools()
        
        # Known actions mapping
        self.known_actions = {
            "search_places": search_places,
            "search_multiple_keywords": search_multiple_keywords,
            "get_place_details": get_place_details
        }

        # Agent state for tracking requirements
        self.required_places = 0
        self.current_results_count = 0
    
    def _get_default_system_prompt(self) -> str:
        """Default system prompt following thought-action-observation pattern."""
        return """
            You are a places and attractions research specialist. You run in a loop of Thought, Action, Observation.
            At the end of the loop you output an Answer.
            
            Use Thought to describe your reasoning about the places and attractions research.
            Use Action to run one of the actions available to you.
            Observation will be the result of running those actions.
            
            INTEREST MAPPING RULES:
            Before processing any interests, map them to the following standardized categories:
            - tourist_attraction
            - food
            - cafe
            - bar
            - bakery
            - park
            - museum
            - shopping_mall
            - lodging
            
            Mapping examples:
            - "parks" to park
            - "museums" to museum
            - "family" to tourist_attraction
            - "educational" to museum or tourist_attraction
            - "gardens" to park
            - "nature" to park
            - "culture" to museum or 'cultural food'
            - "dining" to food
            - "coffee" to cafe
            - "shopping" to shopping_mall
            - "accommodation" to lodging
            
            Special handling for food-related interests:
            If an interest cannot be classified into the above categories but refers to food options 
            (like "vegetarian", "halal", "vegan", "local cuisine"), rewrite it as "[interest] food".
            Examples:
            - "vegetarian" → "vegetarian food"
            - "halal" → "halal food"
            - "local cuisine" → "local food"
            
            Your available actions are:
            
            search_places:
            e.g. search_places: {"interests": ["park", "museum"], "accommodation_location": {"lat": 1.3521, "lng": 103.8198, "neighborhood": "Orchard Road"}, "radius": 20000}
            Searches for attractions based on mapped interests and accommodation location
            This radius should increase if search places is not enough
            
            get_place_details:
            e.g. get_place_details: {"place_id": "ChIJ...", "fields": ["name", "rating", "reviews"]}
            Gets detailed information about a specific place with comprehensive data including coordinates, accessibility, and match scoring
            
            find_nearby_places:
            e.g. find_nearby_places: {"lat": 1.2966, "lng": 103.7764, "type": "tourist_attraction", "radius": 2000}
            Finds attractions near specific coordinates (used to find food options after attractions are clustered into groups)
            
            Focus on providing comprehensive attraction recommendations based on user interests and accommodation location.
            Always consider practical factors like location, ratings, opening hours, accessibility needs, and user preferences.
            Calculate required places based on pace and duration, and expand search if insufficient results are found.
            """.strip()

    def _define_places_tools(self) -> List[Dict]:
        """Define Google Places API tools."""
        return [
            {
                "type": "function",
                "function": {
                    "name": "search_places",
                    "description": "Search for places and attractions based on user interests and accommodation location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "interests": {
                                "type": "array",
                                "items": {"type": "string"},
                                "description": "User interests like 'parks', 'culture', 'family', 'museums', 'educational', 'food', 'nature', 'shopping'"
                            },
                            "accommodation_location": {
                                "type": "object",
                                "properties": {
                                    "lat": {"type": "number", "description": "Latitude of accommodation"},
                                    "lng": {"type": "number", "description": "Longitude of accommodation"},
                                    "neighborhood": {"type": "string", "description": "Neighborhood name (optional)"}
                                },
                                "required": ["lat", "lng"],
                                "description": "Accommodation location with coordinates and optional neighborhood"
                            },
                            "radius": {
                                "type": "integer",
                                "description": "Search radius in meters (default: 20000, increases if not enough results)",
                                "default": 20000
                            },
                            "min_rating": {
                                "type": "number",
                                "description": "Minimum rating filter (1.0-5.0)",
                                "default": 4.0
                            },
                            "pace": {
                                "type": "string",
                                "description": "Trip pace to calculate required places",
                                "enum": ["slow", "relaxed", "moderate", "active", "standard", "fast", "intensive"],
                                "default": "moderate"
                            },
                            "duration_days": {
                                "type": "integer",
                                "description": "Trip duration in days",
                                "default": 1
                            }
                        },
                        "required": ["interests", "accommodation_location"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "get_place_details",
                    "description": "Get comprehensive detailed information about a specific place",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "place_id": {
                                "type": "string",
                                "description": "Google Places API place_id"
                            },
                            "fields": {
                                "type": "array",
                                "items": {"type": "string"},
                                "description": "Specific fields to retrieve",
                                "default": ["name", "rating", "formatted_address", "opening_hours", "price_level", "reviews", "coordinates", "accessibility"]
                            }
                        },
                        "required": ["place_id"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "find_nearby_places",
                    "description": "Find attractions near specific coordinates (used for food options after clustering)",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "lat": {
                                "type": "number",
                                "description": "Latitude coordinate"
                            },
                            "lng": {
                                "type": "number", 
                                "description": "Longitude coordinate"
                            },
                            "place_type": {
                                "type": "string",
                                "description": "Type of place to search for",
                                "enum": ["tourist_attraction", "restaurant", "shopping_mall", "park", "museum", "temple", "food"],
                                "default": "tourist_attraction"
                            },
                            "radius": {
                                "type": "integer",
                                "description": "Search radius in meters",
                                "default": 2000
                            }
                        },
                        "required": ["lat", "lng"]
                    }
                }
            }
        ]

    def __call__(self, message: str) -> str:
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self) -> str:
        """Execute with function calling support."""
        response = self.client.chat.completions.create(
            model=self.model_name,
            temperature=self.temperature,
            messages=self.messages,
            tools=self.available_tools,
            tool_choice="auto"
        )
        
        message = response.choices[0].message
        
        # Handle function calls if present
        if message.tool_calls:
            # Add assistant message with tool calls
            self.messages.append({
                "role": "assistant",
                "content": message.content,
                "tool_calls": message.tool_calls
            })
            
            # Execute tool calls
            for tool_call in message.tool_calls:
                tool_result = self._execute_tool_call(tool_call)
                
                # Add tool result to messages
                self.messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(tool_result, default=str)
                })
            
            # Get final response after tool execution
            final_response = self.client.chat.completions.create(
                model=self.model_name,
                temperature=self.temperature,
                messages=self.messages
            )
            
            return final_response.choices[0].message.content
        
        return message.content
    
    def _execute_tool_call(self, tool_call) -> Dict[str, Any]:
        """Execute tool call and return results."""
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        
        print(f"RESEARCH AGENT: Executing {tool_name} with args: {tool_args}")
        
        if tool_name in self.known_actions:
            return self.known_actions[tool_name](**tool_args)
        else:
            return {"error": f"Unknown tool: {tool_name}"}

    # =====================================
    # AGENT INTERNAL LOGIC
    # =====================================

    def calculate_required_places(self, pace: str, duration_days: int) -> int:
        """
        Agent's internal goal-setting: Calculate required number of places.
        This sets the target for the agent to achieve through tool usage.
        No artificial limits - agent will expand search radius to meet the target.
        """
        try:
            pace_mapping = {
                "slow": 2,
                "relaxed": 4,
                "moderate": 4,
                "active": 6,
                "standard": 6,
                "fast": 8,
                "intensive": 8
            }
            
            multiplier = float(os.getenv("MAX_PLACES_MULTIPLIER", "2"))
            pace_value = pace_mapping.get(pace.lower(), 6)
            required_places = int(pace_value * duration_days * multiplier)
            
            # Only enforce minimum, no maximum cap
            required_places = max(required_places, 5)  # Minimum 5 places
            
            print(f"RESEARCH AGENT: Goal set - {required_places} places for {pace} pace over {duration_days} days")
            return required_places
            
        except (ValueError, TypeError, AttributeError):
            print("RESEARCH AGENT: Failed to calculate required places, using default: 15")
            return 15

ImportError: cannot import name 'search_places' from 'tools' (c:\Users\weiwe\Repository\Retrieval Agent\tools.py)

In [7]:
# Load input data
input_file = 'inputs/garden_only_input.json'
input_data = load_input_file(input_file)

# Extract interests and accommodation_location from input_data
interests = input_data.get('optional', {}).get('interests', [])
accommodation_location = input_data.get('optional', {}).get('accommodation_location', {})

# Instantiate the agent
agent = PlacesResearchAgent()

# Compose a message to trigger search_places
message = f"search_places: {{\"interests\": {interests}, \"accommodation_location\": {accommodation_location}, \"radius\": 20000}}"

# Run the agent
result = agent(message)
print(result)

2025-09-26 18:12:55,261 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-26 18:12:55,293 - googlemaps.client - INFO - API queries_quota: 60


RESEARCH AGENT: Executing search_places with args: {'interests': ['gardens'], 'accommodation_location': {'lat': 1.3294, 'lng': 103.8021, 'neighborhood': 'Bukit Timah'}, 'radius': 20000}


2025-09-26 18:13:22,483 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Here are some garden attractions near Bukit Timah, Singapore, that you might find interesting:

1. **Gardens by the Bay**
   - Location: 18 Marina Gardens Dr, Singapore
   - Rating: 4.7 (150,798 reviews)
   - Description: A futuristic park with iconic Supertree structures, offering a variety of themed gardens and attractions.
   - Open Now

2. **Singapore Botanic Gardens**
   - Location: 1 Cluny Rd, Singapore
   - Rating: 4.7 (44,869 reviews)
   - Description: A UNESCO World Heritage Site, this garden features a wide range of plant species and themed gardens.
   - Open Now

3. **Flower Dome**
   - Location: 18 Marina Gardens Dr, Singapore
   - Rating: 4.7 (25,244 reviews)
   - Description: A cooled conservatory at Gardens by the Bay showcasing exotic plants from around the world.
   - Open Now

4. **Cloud Forest**
   - Location: 18 Marina Gardens Dr, Singapore
   - Rating: 4.8 (29,889 reviews)
   - Description: Another cooled conservatory at Gardens by the Bay, featuring a 35-meter tal

In [8]:
def run_agentic_workflow(self, input_data):
    """Run agent workflow: calculate required places and fetch attractions."""
    # Extract parameters
    interests = input_data.get('optional', {}).get('interests', [])
    accommodation_location = input_data.get('optional', {}).get('accommodation_location', {})
    pace = input_data.get('pace', 'moderate')
    duration_days = input_data.get('duration_days', 1)
    radius = 20000
    min_rating = 4.0
    
    # Calculate required places
    required_places = self.calculate_required_places(pace, duration_days)
    print(f"Agent will fetch at least {required_places} attractions.")
    
    # Fetch places using search_places tool
    results = self.known_actions['search_places'](interests, accommodation_location, radius, min_rating)
    
    # Truncate to required_places if needed
    if len(results) > required_places:
        results = results[:required_places]
    
    print(f"Found {len(results)} places:")
    for place in results:
        print(f"{place.get('name', 'Unknown')} (Rating: {place.get('rating', 'N/A')})")
    return results